In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup      #for html removal       
import re                        #for number removal
import nltk                      #for stopwords removal
import nltk.data
from gensim.models import Word2Vec

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.stem import PorterStemmer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.

download from directory

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.') #lists all downloadable files on server

In [ ]:
train=pd.read_csv('../input/labeledTrainData.tsv',delimiter="\t", quoting=3,header=0)
test = pd.read_csv("../input/testData.tsv", header=0, delimiter="\t",quoting=3 )
unl_train=pd.read_csv("../input/unlabeledTrainData.tsv", header=0, delimiter="\t",quoting=3)
print("shape of train:",train.shape)
print("shape of test:",test.shape)
print("shape of unl_train:",unl_train.shape)

Getting all filters in a function

In [ ]:
def rev2words(rev):
    #making beautyful/removing markups and tags
    words=BeautifulSoup(rev).get_text()
    #removing digits,symbols,and everything except plain letters
    letters = re.sub("[^a-zA-Z0-9!?'-]", " ", words)
    lemma=WordNetLemmatizer()
    words_arr=[lemma.lemmatize(w) for w in word_tokenize(str(letters).lower())]
    
    stop_words=set(stopwords.words("english"))
    meaningful_words=[w for w in words_arr if w not in stop_words]
    return " ".join(meaningful_words)

In [ ]:
train['review'][0]

In [ ]:
rev2words(train['review'][0])

In [ ]:
num_reviews = train["review"].size
clean_train_reviews = []

print ("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message,for progress
    if( (i+1)%1000 == 0 ):
        sys.stdout.write('\r'+"Review %d of %d\t" % (i+1, num_reviews))                                                                
    clean_train_reviews.append( rev2words( train['review'][i] ))

In [ ]:
numOfRev=len(test)
clean_test_reviews=[]
print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,numOfRev):
    if( (i+1) % 1000 == 0 ):
        sys.stdout.write('\r'+"Review %d of %d\t" % (i+1, numOfRev))
    clean_review = rev2words( test["review"][i] )
    clean_test_reviews.append( clean_review )

numOfRev=len(unl_train)
clean_unl_reviews=[]
print("Cleaning and parsing the unlabled set movie reviews...\n")
for i in range(0,numOfRev):
    if( (i+1) % 1000 == 0 ):
        sys.stdout.write('\r'+"Review %d of %d\t" % (i+1, numOfRev))
    clean_review = rev2words( test["review"][i] )
    clean_unl_reviews.append( clean_review )

In [ ]:
#creating bag of words

from sklearn.feature_extraction.text import CountVectorizer              #Importing Vectorizer
vectorizer= CountVectorizer(analyzer='word',max_features=2500)

train_data_features = vectorizer.fit_transform(clean_train_reviews)     #Vectorizing training Data
train_data_features = train_data_features.toarray() 

test_data_features = vectorizer.transform(clean_test_reviews)            #Vectorize Test Data
test_data_features = test_data_features.toarray()
import pickle
pickle.dump(vectorizer,open("vectorizer.pickle","wb"))
print(type(vectorizer))

In [ ]:
train_data_features.shape

In [ ]:

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(train_data_features)       #  TFIDF
messages_tfidf = tfidf_transformer.transform(train_data_features)
test_tfidf=tfidf_transformer.transform(test_data_features)

from sklearn.svm import SVC, LinearSVC
linear_svc = LinearSVC()
linear_svc.fit(messages_tfidf, train['sentiment'])                  # SVM
pred = linear_svc.predict(test_tfidf)

In [ ]:
#This is a singularity approach
sample_rev='"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally starts is only on for 20 minutes or so excluding the Smooth Criminal sequence and Joe Pesci is convincing as a psychopathic all powerful drug lord. Why he wants MJ dead so bad is beyond me. Because MJ overheard his plans? Nah, Joe Pesci\'s character ranted that he wanted people to know it is he who is supplying drugs etc so i dunno, maybe he just hates MJ\'s music.<br /><br />Lots of cool things in this like MJ turning into a car and a robot and the whole Speed Demon sequence. Also, the director must have had the patience of a saint when it came to filming the kiddy Bad sequence as usually directors hate working with one kid let alone a whole bunch of them performing a complex dance scene.<br /><br />Bottom line, this movie is for people who like MJ on one level or another (which i think is most people). If not, then stay away. It does try and give off a wholesome message and ironically MJ\'s bestest buddy in this movie is a girl! Michael Jackson is truly one of the most talented people ever to grace this planet but is he guilty? Well, with all the attention i\'ve gave this subject....hmmm well i don\'t know because people can be different behind closed doors, i know this for a fact. He is either an extremely nice but stupid guy or one of the most sickest liars. I hope he is not the latter."'
clean_sample_rev=rev2words(sample_rev)
sample_features=vectorizer.transform([clean_sample_rev])
sample_features=sample_features.toarray()

sample_tfidf=tfidf_transformer.transform(sample_features)
linear_svc.predict(sample_tfidf)

In [ ]:
import pickle
#pickle.dump(tfidf_transformer, open("tfidf_transformer.pickle", "wb"))
#pickle.dump(linear_svc,open("linear_svc.pickle","wb"))
pickle.dump(vectorizer,open("count_vec.pickle","wb"))

In [ ]:
acc_linear_svc = round(linear_svc.score(messages_tfidf, train['sentiment']) * 100, 2)
acc_linear_svc

In [ ]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":pred} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )